In [1]:
import pandas as pd

import torch
from sentence_transformers import SentenceTransformer

import faiss
import numpy as np
import pickle
from pathlib import Path



In [2]:
df = pd.read_csv("../data/input.csv")
df.head()

,Unnamed: 0,title,img_captions_str,meme_captions_str,metaphors_str,url,input
0,0,For real though,Person in Spider Man outfit gives a lecture on...,Meme poster is frustrated about the format of ...,"Spider Man outfit: Meme poster, a lecture: com...",https://i.redd.it/m16dhaqyply21.jpg,TEXT: For real though; Person in Spider Man ou...
1,1,And that's a fact,"Two dogs carry a white flag, Identical chubby ...",Meme poster is saying that searching Google pl...,"Two dogs: Meme poster, a white flag: PSA, carr...",https://i.redd.it/z9oh7ligb0i31.jpg,TEXT: And that's a fact; Two dogs carry a whit...
2,2,It was horrible,"man is very dissatisfied with what he sees, Qu...",Meme poster is feeling sad.,"man: Meme poster, sees: expressing different r...",https://i.redd.it/yves3izsbsj31.jpg,TEXT: It was horrible; man is very dissatisfie...
3,3,This is why Reddit is better,A man that is crying and a man that is laughin...,Meme poster is saying that Reddit is more fun ...,"A man: Meme poster, a man: Meme poster, his go...",https://i.redd.it/y594n8exi6k31.jpg,TEXT: This is why Reddit is better; A man that...
4,4,The Area 51 raid is still happening right?,"a cop is chasing a thief with a shopping cart,...",Meme poster is making fun of the Area 51 raid.,"a cop: Meme poster, a thief: not being bothere...",https://i.redd.it/4hrn18t4lck31.jpg,TEXT: The Area 51 raid is still happening righ...


In [3]:
df = df.reset_index()
df = df.rename(columns={"index":"id"})
df['id'] = df.index 
df["id"]

0          0
1          1
2          2
3          3
4          4
        ... 
5813    5813
5814    5814
5815    5815
5816    5816
5817    5817
Name: id, Length: 5818, dtype: int64

In [4]:
df

,id,Unnamed: 0,title,img_captions_str,meme_captions_str,metaphors_str,url,input
0,0,0,For real though,Person in Spider Man outfit gives a lecture on...,Meme poster is frustrated about the format of ...,"Spider Man outfit: Meme poster, a lecture: com...",https://i.redd.it/m16dhaqyply21.jpg,TEXT: For real though; Person in Spider Man ou...
1,1,1,And that's a fact,"Two dogs carry a white flag, Identical chubby ...",Meme poster is saying that searching Google pl...,"Two dogs: Meme poster, a white flag: PSA, carr...",https://i.redd.it/z9oh7ligb0i31.jpg,TEXT: And that's a fact; Two dogs carry a whit...
2,2,2,It was horrible,"man is very dissatisfied with what he sees, Qu...",Meme poster is feeling sad.,"man: Meme poster, sees: expressing different r...",https://i.redd.it/yves3izsbsj31.jpg,TEXT: It was horrible; man is very dissatisfie...
3,3,3,This is why Reddit is better,A man that is crying and a man that is laughin...,Meme poster is saying that Reddit is more fun ...,"A man: Meme poster, a man: Meme poster, his go...",https://i.redd.it/y594n8exi6k31.jpg,TEXT: This is why Reddit is better; A man that...
4,4,4,The Area 51 raid is still happening right?,"a cop is chasing a thief with a shopping cart,...",Meme poster is making fun of the Area 51 raid.,"a cop: Meme poster, a thief: not being bothere...",https://i.redd.it/4hrn18t4lck31.jpg,TEXT: The Area 51 raid is still happening righ...
...,...,...,...,...,...,...,...,...
5813,5813,5818,Posting memes until I get my master's degree d...,The middle aged somewhat bolding man is at a s...,Poster is low on funds towards end of the month.,somewhat bolding man: Meme poster,https://i.redd.it/p5t3m0ujt24a1.jpg,TEXT: Posting memes until I get my master's de...
5814,5814,5819,*pretends to think*,A woman is deep in thought throughout the day.,The poster is pretending to think when they ar...,A woman: the poster,https://i.redd.it/yyru9fqbu24a1.jpg,TEXT: *pretends to think*; A woman is deep in ...
5815,5815,5820,overreaction indeed,Captain America talks to a man in a elevator t...,The meme poster exposed that he could fight wi...,"Captain America: Meme poster, talks: exposing",https://i.redd.it/n2qnaoztu24a1.jpg,TEXT: overreaction indeed; Captain America tal...
5816,5816,5821,Anime watchers be like,two drawn faces face to face and right one is ...,Meme poster is trying to convey that Anime fan...,two drawn faces: anime fans and not fans,https://farm66.staticflickr.com/65535/52761413...,TEXT: Anime watchers be like; two drawn faces ...


In [5]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
model = model.to("mps")

In [6]:
embeddings = model.encode(df["input"].to_list(), show_progress_bar=True)

Batches:   0%|          | 0/182 [00:00<?, ?it/s]

In [8]:
print(f"Shape of one embedding{embeddings[0].shape}")

Shape of one embedding(768,)


In [15]:
from sentence_transformers import util
import pandas as pd
prompt = "Dustbin"
prompt_embedding = model.encode(prompt, convert_to_tensor=True)
hits = util.semantic_search(prompt_embedding,embeddings, top_k=5)
hits = pd.DataFrame(hits[0], columns=['corpus_id', 'score'])

In [16]:
desired_ids = hits["corpus_id"]

In [17]:
filtered_df = df.loc[df['id'].isin(desired_ids)]
filtered_df.head()

,id,Unnamed: 0,title,img_captions_str,meme_captions_str,metaphors_str,url,input
188,188,188,Society is strange,A large wolf attacks a city with a green blob ...,The meme poster thinks it is funny that the ga...,a green blob: the garbage collector,https://i.redd.it/j5ydxwew5o861.jpg,TEXT: Society is strange; A large wolf attacks...
2944,2944,2948,Crying hacker noises,an image of a shelf with lots of papers in it,The meme poster's grandpa is storing lots and ...,"a shelf: Data center, papers: Data",https://i.redd.it/bollrng5030a1.jpg,TEXT: Crying hacker noises; an image of a shel...
4109,4109,4114,Forgot the dumpster,fire room,Meme poster is trying to convey that the comme...,fire room: comments section,https://i.redd.it/xtz2v9omoc1a1.jpg,TEXT: Forgot the dumpster; fire room; Meme pos...
4457,4457,4462,Environmentally friendly litterbug,A man on suit looking tiredly,The meme poster is trying to convey that they ...,"A man: Meme poster, suit: Meme poster",https://i.redd.it/3ijlnsm9el1a1.jpg,TEXT: Environmentally friendly litterbug; A ma...
4772,4772,4777,That's why I hate apple.,A mouse is trying to escape from the dustbin a...,Meme poster is trying to convey that Apple is ...,"A mouse: apple, the dustbin: your data",https://i.redd.it/lgvmd586lo2a1.jpg,TEXT: That's why I hate apple.; A mouse is try...


In [10]:
# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, df.id.values)

print(f"Number of vectors in the Faiss index: {index.ntotal}")

Number of vectors in the Faiss index: 5818


In [11]:
df.iloc[5415, 1]

5420

In [ ]:
D, I = index.search(np.array([embeddings[5415]]), k=3)
print(f'L2 distance: {D.flatten().tolist()}\n\nMemes: {I.flatten().tolist()}')

In [ ]:
D, I = index.search(np.array([embeddings[5415]]), k=3)